In [ ]:
# https://learn.datacamp.com/courses/software-engineering-for-data-scientists-in-python
# https://learn.datacamp.com/courses/unit-testing-for-data-science-in-python

In [9]:
from data.Stock import Stock
from data.Options import Options
from data.TS import series_to_supervised

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd
from keras.models import Model
from keras.layers import LSTM,Dense,Dropout,Input

Using TensorFlow backend.


In [26]:
stock = Stock("AAPL")
optionHandler = Options("AAPL")

2010-01-04 00:00:00 <class 'pandas._libs.tslibs.timestamps.Timestamp'>
Index(['Open', 'High', 'Low', 'Close', 'Adj_Close', 'Volume'], dtype='object')
RETRIEVING DATA FROM CSV
(1011, 10) (1052, 10)


In [46]:
days = 10

puts,calls = optionHandler.get_puts_calls()
puts = series_to_supervised(puts,days,1)
calls = series_to_supervised(calls,days,1)
target = stock.get_data()['Adj_Close']
target = pd.Series(target).pct_change(days).shift(-days).dropna().values * 100
features_futureP = puts.iloc[-days:,:]
features_futureC = calls.iloc[-days:,:]
puts,calls = puts.iloc[:-days,:],calls.iloc[:-days,:]

In [ ]:
# Use wallstreet package blackandscholes for calculations of greeks.
# MATRIX PROFILING

In [47]:
puts.index
# https://github.com/munozalexander/Stock-Prediction
# https://github.com/ltnguyen14/Quant_stock
# https://github.com/jackluo/py-quantmod
# https://github.com/achillesrasquinha/bulbea

MultiIndex([('2020-06-25', 16),
            ('2020-06-25', 17),
            ('2020-06-25', 18),
            ('2020-06-25', 19),
            ('2020-06-25', 20),
            ('2020-06-25', 21),
            ('2020-06-25', 22),
            ('2020-06-25', 23),
            ('2020-06-25', 24),
            ('2020-06-25', 25),
            ...
            ('2022-09-15', 11),
            ('2022-09-15', 12),
            ('2022-09-15', 13),
            ('2022-09-15', 14),
            ('2022-09-15', 15),
            ('2022-09-15', 16),
            ('2022-09-15', 17),
            ('2022-09-15', 18),
            ('2022-09-15', 19),
            ('2022-09-15', 20)],
           length=758)

10 if x>= 10
9  if x>=9 and x< 10
8  if x>=8 and x<9
7  if x>=7 and x<8
6  if x>=6 and x<7
5  if x>=5 and x<6
4  if x>=4 and x<5
3  if x>=3 and x<4
2  if x>=2 and x<3
1  if x>=1 and x<2
0  if x>=0 and x<1
-1 if x>=-1 and x<0
-2 if x>=-2 and x<-1
-3 if x>=-3 and x<-2
-4 if x>=-4 and x<-3
-5 if x>=-5 and x<-4
-6 if x>=-6 and x<-5
-7 if x>=-7 and x<-6
-8 if x>=-8 and x<-7
-9  if x>=-9 and x<-8
-10 if x>-10 and x<-9
-11 if x<=-10

In [28]:
# target = target * 100
# target = pd.Series(target)

target = [10 if x>=10 else x for x in target]
target = [-11 if x<=-10 else x for x in target]
target = [np.floor(x) if x>-10 and x<10 else x for x in target]

In [29]:
from sklearn.model_selection import train_test_split

In [34]:
target_size = len(target) - days
X_trainP, X_testP = train_test_split(puts,test_size=0.2,shuffle=False)
X_trainC, X_testC = train_test_split(calls,test_size=0.2,shuffle=False)
y_train, y_test = train_test_split(target,test_size=0.2,shuffle=False)
print("Puts Train Test {} {} \n Calls Train Test {} {} \n Targets train test {} {}".format(\
                                     X_trainP.shape,
                                     X_testP.shape,
                                     X_trainC.shape,
                                     X_testC.shape,
                                     len(y_train),
                                     len(y_test),
                                    ))


Puts Train Test (606, 110) (152, 110) 
 Calls Train Test (695, 110) (174, 110) 
 Targets train test 2103 526


In [45]:
def get_unique_list(df):
    index_dates = [x[0][0] for x in X_trainP.index]
    set1 = set(index_dates)
    return list(set1)
uniques = get_unique_list(X_trainP)
print(uniques)

['2']


In [31]:
y_train, y_test = np.array(y_train),np.array(y_test)
X_trainP, X_trainC = np.array(X_trainP), np.array(X_trainC)
X_testP,  X_testC  = np.array(X_testP), np.array(X_testC)

In [32]:
model1_inputs = tf.keras.Input(shape=(X_trainP.shape[0],X_trainP.shape[1]))
model1_layer1 = tf.keras.layers.LSTM(16)(model1_inputs)
# 21 classes
model1_layer2 = tf.keras.layers.Dense(21, activation='softmax')(model1_layer1)

model2_inputs = tf.keras.Input(shape=(X_trainC.shape[0],X_trainC.shape[1]))
model2_layer1 = tf.keras.layers.LSTM(16)(model2_inputs)
# 21 classes
model2_layer2 = tf.keras.layers.Dense(21,activation='softmax')(model2_layer1)


merged = tf.keras.layers.add([model1_layer2,model2_layer2])
model = tf.keras.Model(inputs=[model1_inputs,model2_inputs],outputs=merged)
model.compile('adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [33]:
history = model.fit(\
                   x=[X_trainP,X_trainC],
                   y=y_train,
                   validation_data=([X_testP,X_testC], y_test),
                   shuffle=False)

ValueError: Error when checking input: expected input_5 to have 3 dimensions, but got array with shape (606, 110)